In [ ]:
from glob import glob
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
import copy

import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.io import read_image
import torch.optim as optim
import torchvision.models as models
import torch.nn as nn

from torchvision.models.mnasnet import mnasnet0_5, mnasnet0_75, mnasnet1_0, mnasnet1_3 

# SAMPLE CODE USED FROM TORCHVISION: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

In [ ]:
train_split = 0.8

# input_size = 224
input_size = 224

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()

NUM_EPOCHS = 25
BATCH_SIZE = 64

WEIGHT_DECAY = 0.01
LR = 0.003

HIDDEN_SIZE = 1000

In [ ]:
DATA_PATH = "."

In [ ]:
classes = (
    'Unknown', 'Compacts', 'Sedans'
)
NUM_CLASSES = len(classes)

In [ ]:
# Data loader
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, label_info, transform=None, target_transform=None):
        self.label_info = label_info
        self.shuffle_data()
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def shuffle_data(self):
        self.label_info = self.label_info.sample(frac=1)
        self.img_labels = self.label_info["label"].values.tolist()
        self.img_files = self.label_info["guid/image"].values.tolist()

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx]) + "_image.jpg"
        image = read_image(img_path).float()
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
# Shuffle, Split Data into train/val
label_info = pd.read_csv(os.path.join(DATA_PATH, "trainval", "labels.csv"))
label_info = label_info.sample(frac=1)

split_point = int(train_split * label_info["label"].size) 

img_dir = os.path.join(DATA_PATH, "trainval")

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

training_data = CustomImageDataset(img_dir, label_info[:split_point], transform=data_transforms["train"])
val_data = CustomImageDataset(img_dir, label_info[split_point:], transform=data_transforms["val"])

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

dataloaders = {
    "train": train_dataloader,
    "val": val_dataloader
}

In [ ]:
# Data loader
def save_data(model, loc="./submission.csv"):
    guid_images = []
    pred_labels = []

    # Create Submission
    with torch.no_grad():
        # Loop over files
        for folder in os.listdir("./test"):
            files = os.listdir("./test/" + folder)
            for file in files:
                if file.endswith(".jpg"):
                    image_id = file.split("_")[0]
                    img_path = "./test/" + folder + "/" + file
                    image = read_image(img_path).float()
                    image = data_transforms["val"](image)
                    image = image.unsqueeze(0).to(device)
                    outputs = model(image)
                    _, preds = torch.max(outputs, 1)
                    pred_labels.append(preds.detach().cpu().numpy()[0])
                    guid_images.append(folder + "/" + image_id)


    d = {"guid/image": guid_images, "label": pred_labels}
    new_dataframe = pd.DataFrame(data=d)
    new_dataframe.to_csv(loc, index=False)

In [ ]:
# Train
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history


In [ ]:
def create_model():
    # Create model, optimizer
    model = models.resnet50(pretrained=True)
    new_end = nn.Sequential(
        nn.Linear(in_features=2048, out_features=HIDDEN_SIZE, bias=True),
        nn.ReLU(inplace=True),
        nn.Linear(in_features=HIDDEN_SIZE, out_features=HIDDEN_SIZE, bias=True),
        nn.ReLU(inplace=True),
        nn.Linear(in_features=HIDDEN_SIZE, out_features=NUM_CLASSES)
    )
    model.fc = new_end
    
#     model = mnasnet1_3()
#     new_end = nn.Sequential(
#     nn.Linear(in_features=1280, out_features=HIDDEN_SIZE, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Linear(in_features=HIDDEN_SIZE, out_features=HIDDEN_SIZE, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Linear(in_features=HIDDEN_SIZE, out_features=NUM_CLASSES)
# )
#     model.classifier = new_end

    for param in model.parameters():
        param.requires_grad = False
    for param in model.fc.parameters():
        param.requires_grad = True

    model = model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, momentum=0.9, nesterov=True)
    return model, optimizer

In [ ]:
# Basic cross entropy
model, optimizer = create_model()
mnasnet_model, hist = train_model(model, dataloaders, criterion, optimizer, num_epochs=NUM_EPOCHS)
save_data(mnasnet_model, loc="./mnasnet_submission.csv")

Epoch 0/24
----------


/home/tigeriv/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 0.8295 Acc: 0.6288
val Loss: 0.7753 Acc: 0.6647

Epoch 1/24
----------
train Loss: 0.7233 Acc: 0.7058
val Loss: 0.6985 Acc: 0.7056

Epoch 2/24
----------
train Loss: 0.6506 Acc: 0.7309
val Loss: 0.6529 Acc: 0.7142

Epoch 3/24
----------
train Loss: 0.5950 Acc: 0.7595
val Loss: 0.5852 Acc: 0.7611

Epoch 4/24
----------
train Loss: 0.5517 Acc: 0.7833
val Loss: 0.5516 Acc: 0.7710

Epoch 5/24
----------
train Loss: 0.5293 Acc: 0.7856
val Loss: 0.5866 Acc: 0.7518

Epoch 6/24
----------
train Loss: 0.5029 Acc: 0.8004
val Loss: 0.5168 Acc: 0.7736

Epoch 7/24
----------
train Loss: 0.5019 Acc: 0.7994
val Loss: 0.5502 Acc: 0.7545

Epoch 8/24
----------
train Loss: 0.4942 Acc: 0.8064
val Loss: 0.5097 Acc: 0.7881

Epoch 9/24
----------
train Loss: 0.4743 Acc: 0.8110
val Loss: 0.4955 Acc: 0.7868

Epoch 10/24
----------
train Loss: 0.4752 Acc: 0.8145
val Loss: 0.4833 Acc: 0.7921

Epoch 11/24
----------
train Loss: 0.4679 Acc: 0.8110
val Loss: 0.4725 Acc: 0.7974

Epoch 12/24
----------
t

In [ ]:
# Classes weight
label_info = pd.read_csv(os.path.join(DATA_PATH, "trainval", "labels.csv"))
inds, nums = np.unique(label_info["label"], return_counts=True)
ind_weights = np.zeros(NUM_CLASSES)
for i in range(inds.shape[0]):
    ind_weights[inds[i]] = nums[i]
label_weights = torch.tensor(ind_weights / np.sum(ind_weights), device=device).float()
criterion = nn.CrossEntropyLoss(weight=label_weights)
model, optimizer = create_model()
weighted_model, hist = train_model(model, dataloaders, criterion, optimizer, num_epochs=NUM_EPOCHS)
save_data(weighted_model, loc="./weighted_submission.csv")

Epoch 0/24
----------
train Loss: 0.5522 Acc: 0.6256
val Loss: 0.4936 Acc: 0.6337

Epoch 1/24
----------
train Loss: 0.4652 Acc: 0.6370
val Loss: 0.4584 Acc: 0.6403

Epoch 2/24
----------
train Loss: 0.4354 Acc: 0.6738
val Loss: 0.4378 Acc: 0.6878

Epoch 3/24
----------
train Loss: 0.4182 Acc: 0.6964
val Loss: 0.4291 Acc: 0.7089

Epoch 4/24
----------
train Loss: 0.4057 Acc: 0.7058
val Loss: 0.4126 Acc: 0.7083

Epoch 5/24
----------
train Loss: 0.3918 Acc: 0.7139
val Loss: 0.4066 Acc: 0.7221

Epoch 6/24
----------
train Loss: 0.3848 Acc: 0.7266
val Loss: 0.3935 Acc: 0.7168

Epoch 7/24
----------
train Loss: 0.3782 Acc: 0.7276
val Loss: 0.3874 Acc: 0.7149

Epoch 8/24
----------
train Loss: 0.3734 Acc: 0.7308
val Loss: 0.3826 Acc: 0.7175

Epoch 9/24
----------
train Loss: 0.3666 Acc: 0.7344
val Loss: 0.3956 Acc: 0.7419

Epoch 10/24
----------
train Loss: 0.3614 Acc: 0.7428
val Loss: 0.3786 Acc: 0.7432

Epoch 11/24
----------
train Loss: 0.3675 Acc: 0.7374
val Loss: 0.3731 Acc: 0.7419

Ep

In [ ]:
from sklearn import svm
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

mnasnet_model.fc[4] = Identity()
model = mnasnet_model
pred_feats = []
with torch.no_grad():
    # Loop over files
    for folder in os.listdir("./trainval"):
        files = os.listdir("./trainval/" + folder)
        for file in files:
            if file.endswith(".jpg"):
                image_id = file.split("_")[0]
                img_path = "./test/" + folder + "/" + file
                image = read_image(img_path).float()
                image = data_transforms["val"](image)
                image = image.unsqueeze(0).to(device)
                outputs = model(image)
                _, preds = torch.max(outputs, 1)
                pred_feats.append(preds.detach().cpu().numpy())
                
                
lin_clf = svm.LinearSVC()
lin_clf.fit(X, Y)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
print(mnasnet_model.fc[4])

Linear(in_features=1000, out_features=3, bias=True)
